# Scrapy — Data Extraction & Pipelines (45 Worked Examples)

Sections (15 examples each):
1) **Crawl rules and link following**
2) **Exporting data (JSON, CSV, XML)**
3) **Data pipelines and item processing**

Each example includes:
- **What it is** (markdown)
- **Code snippet** (runnable in a real Scrapy project)
- **Explanation** (markdown)
- **Optimization options** (markdown)

> These cells are templates for a real Scrapy environment (`pip install scrapy`). Networking is disabled here, but the code runs in your local project.


## Section 1 — Crawl rules and link following (15 Examples)

### CrawlSpider with rules
Rule-driven crawling using `CrawlSpider` and `Rule`.

In [ ]:
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

class DocsSpider(CrawlSpider):
    name = "docs"
    allowed_domains = ["example.com"]
    start_urls = ["https://example.com/docs/"]
    rules = (
        Rule(LinkExtractor(allow=r"/docs/page/\d+"), follow=True),
        Rule(LinkExtractor(allow=r"/docs/\d{4}/[\w-]+/"), callback="parse_doc"),
    )
    def parse_doc(self, response):
        yield {
            "title": response.css("h1::text").get(),
            "url": response.url
        }

**Explanation**
CrawlSpider automates link discovery via `LinkExtractor` patterns and applies callbacks to matching pages.

**Optimization options**
- Tighten `allow` regexes to avoid over-crawling; use `deny`/`restrict_xpaths` to narrow scope.

### LinkExtractor allow/deny
Control which links are followed using allow/deny patterns.

In [ ]:
from scrapy.linkextractors import LinkExtractor
lex = LinkExtractor(allow=(r"/category/[\w-]+/",), deny=(r"/logout", r"/admin"))
# In rules: Rule(lex, follow=True)

**Explanation**
`allow` is a whitelist; `deny` blocks unwanted paths. Combine both to avoid noisy sections.

**Optimization options**
- Prefer anchored, specific regex; avoid greedy `.*` patterns.

### restrict_xpaths / restrict_css
Extract links only from a portion of the page.

In [ ]:
Rule(LinkExtractor(restrict_xpaths="//div[@id='content']"), follow=True)

**Explanation**
When pages have navigation/ads, restrict extraction to content areas only.

**Optimization options**
- Reducing link scope lowers noise and crawl volume — faster, cheaper.

### process_links hook
Filter or transform links after extraction.

In [ ]:
def proc_links(self, links):
    return [l for l in links if 'utm_' not in l.url]
rules = (Rule(LinkExtractor(), process_links='proc_links', follow=True),)

**Explanation**
`process_links` receives a list of `Link` objects; return a filtered list.

**Optimization options**
- Use for deduping/tracking removal; keep logic idempotent and cheap.

### process_request hook
Modify each `Request` before scheduling (headers, priority).

In [ ]:
def proc_req(self, request, response):
    request.priority = 10
    request.headers.setdefault(b"Accept-Language", b"en-US")
    return request
rules = (Rule(LinkExtractor(), process_request='proc_req', follow=True),)

**Explanation**
Allows per-link Request customization (priority, headers).

**Optimization options**
- Avoid heavy operations here; it runs for every discovered link.

### response.follow vs urljoin
Prefer `response.follow` for relative links.

In [ ]:
def parse(self, response):
    for href in response.css("a.article::attr(href)").getall():
        yield response.follow(href, callback=self.parse_article)

**Explanation**
`response.follow` keeps cookies/headers and resolves relative URLs.

**Optimization options**
- Use `response.follow` over `response.urljoin(href)` for convenience and correctness.

### Depth limit & crawl order
Control crawl breadth/depth and priority.

In [ ]:
# settings.py
DEPTH_LIMIT = 3
DEPTH_PRIORITY = 1   # BFS-like
SCHEDULER_DISK_QUEUE = "scrapy.squeues.PickleFifoDiskQueue"
SCHEDULER_MEMORY_QUEUE = "scrapy.squeues.FifoMemoryQueue" 

**Explanation**
Depth settings constrain recursion; FIFO queues approximate BFS; LIFO approximates DFS.

**Optimization options**
- Shallow BFS often surfaces site breadth fast for discovery.

### SitemapSpider (rules-like via sitemaps)
Follow links from XML sitemaps.

In [ ]:
from scrapy.spiders import SitemapSpider

class SiteMapDemo(SitemapSpider):
    name = "sitemap_demo"
    sitemap_urls = ["https://example.com/sitemap.xml"]
    def parse(self, response):
        yield {"url": response.url, "title": response.css("title::text").get()}

**Explanation**
`SitemapSpider` reads sitemap URLs and schedules requests directly, avoiding page-level link extraction.

**Optimization options**
- Great for SEO-focused sites; combine with robots rules and rate limits.

### Handle canonical links
Respect canonical hrefs to avoid dupes.

In [ ]:
def parse(self, response):
    canon = response.xpath("//link[@rel='canonical']/@href").get()
    url = canon or response.url
    yield {"url": url}

**Explanation**
Canonical URLs reduce duplicates when the same content appears under multiple paths.

**Optimization options**
- Hash content or use normalized URLs for dedupe in pipelines.

### nofollow and robots meta
Skip links marked nofollow or pages with noindex if you choose.

In [ ]:
# In process_links or parse:
# if 'nofollow' in link.nofollow: skip
# if 'noindex' in response.xpath("//meta[@name='robots']/@content").get(''):
#     return

**Explanation**
While Scrapy doesn’t enforce `nofollow`, you can respect it to be polite.

**Optimization options**
- Track robots directives to align with site policies.

### CB arguments via cb_kwargs
Pass structured data to callbacks cleanly.

In [ ]:
def parse_list(self, response):
    for href in response.css("a.post::attr(href)").getall():
        yield response.follow(href, callback=self.parse_post, cb_kwargs={'from_list': response.url})
def parse_post(self, response, from_list):
    yield {'url': response.url, 'ref': from_list}

**Explanation**
`cb_kwargs` avoids stuffing data in `meta` and gives typed parameters in the signature.

**Optimization options**
- Prefer `cb_kwargs` for small context; keep `meta` for internal flags.

### Errbacks on followed links
Attach errbacks to handle per-request failures.

In [ ]:
for href in response.css('a.post::attr(href)').getall():
    yield response.follow(href, callback=self.parse_post, errback=self.on_err)
def on_err(self, failure):
    self.logger.warning('Failed %s', failure.request.url)

**Explanation**
Per-request errbacks let you log, retry, or route failures.

**Optimization options**
- Design idempotent parse functions so retry is safe.

### Avoid duplicate schedules
Let dupefilter work; include distinguishing params when needed.

In [ ]:
yield scrapy.Request(url, callback=self.parse)              # deduped by URL fingerprint
yield scrapy.Request(url + '?v=2', callback=self.parse)       # different URL -> separate request
# or use dont_filter=True when you must re-request the same URL

**Explanation**
Scrapy fingerprints Requests to avoid duplicates. Add distinguishing query params only when content truly differs.

**Optimization options**
- Avoid `dont_filter=True` unless you know you need a second fetch.

### Handle infinite scroll (API)
Discover next pages via JSON API instead of HTML.

In [ ]:
def parse(self, response):
    api = response.css('div#feed::attr(data-api)').get()
    if api:
        yield response.follow(api, callback=self.parse_api)
def parse_api(self, response):
    data = response.json()
    for item in data['results']:
        yield item
    if data.get('next'): yield response.follow(data['next'], callback=self.parse_api)

**Explanation**
Some sites render via JS but expose JSON endpoints; crawling the API is cleaner and faster.

**Optimization options**
- Cap pagination; validate schema to avoid surprises.

### Politeness and concurrency per domain
Limit concurrency and delay per target domain.

In [ ]:
custom_settings = {
    "CONCURRENT_REQUESTS_PER_DOMAIN": 4,
    "DOWNLOAD_DELAY": 0.25,
    "AUTOTHROTTLE_ENABLED": True,
}

**Explanation**
Per-spider `custom_settings` can keep a crawl polite without changing global settings.

**Optimization options**
- Measure server latency and back off aggressively when needed.

## Section 2 — Exporting data (JSON, CSV, XML) (15 Examples)

### Quick export with -O
Overwrite output using FEEDS (Scrapy 2.1+).

In [ ]:
# Terminal
# scrapy crawl quotes -O items.json   # overwrite JSON
# scrapy crawl quotes -O items.csv    # overwrite CSV


**Explanation**
`-O` flag writes via FEEDS to a file, replacing existing content.

**Optimization options**
- Prefer `-O` for idempotent runs; `-o` appends/updates.

### Multiple outputs via FEEDS
Write JSON and CSV simultaneously.

In [ ]:
# settings.py
FEEDS = {
    "items.json": {"format":"json", "overwrite": True, "indent": 2},
    "items.csv":  {"format":"csv",  "overwrite": True, "encoding":"utf-8"},
}

**Explanation**
The FEEDS dict configures multiple exporters at once.

**Optimization options**
- Use timestamped filenames like `items-%(time)s.json` for unique outputs.

### JSON Lines (streaming)
Write line-delimited JSON (one item per line).

In [ ]:
# Terminal
# scrapy crawl quotes -O items.jl


**Explanation**
JSON Lines is great for streaming/append workflows and large datasets.

**Optimization options**
- Prefer `.jl` when you want incremental processing and easy concatenation.

### XML export
Export to XML via FEEDS.

In [ ]:
# settings.py
FEEDS = {"items.xml": {"format":"xml", "overwrite": True}}

**Explanation**
XML exporter writes a simple hierarchical structure, useful for legacy systems.

**Optimization options**
- Define `FEED_EXPORT_FIELDS` to control field order in XML.

### Field order control
Specify output column order for CSV/XML.

In [ ]:
# settings.py
FEED_EXPORT_FIELDS = ["title","author","tags","url"]

**Explanation**
Consistent field order helps stable diffs and downstream ingestion.

**Optimization options**
- Keep a single place where field order is defined to avoid drift.

### Encoding and newline handling
Set encoding explicitly for CSV/JSON outputs.

In [ ]:
# settings.py
FEEDS = {"items.csv":{"format":"csv","encoding":"utf-8","overwrite":True}}

**Explanation**
Explicit encodings avoid platform surprises (Windows newlines, default ASCII).

**Optimization options**
- Always choose UTF-8 unless you have a legacy requirement.

### Compressed feeds
Write compressed outputs with `.gz` extension.

In [ ]:
# settings.py
FEEDS = {"items.json.gz":{"format":"json","overwrite":True}}

**Explanation**
Scrapy compresses based on filename extension like `.gz`.

**Optimization options**
- Compression reduces storage/network cost; measure CPU tradeoffs.

### Feed storage to S3/FTP
Store outputs remotely via feed URIs.

In [ ]:
# settings.py (S3 requires boto3)
FEEDS = {"s3://my-bucket/items.json":{"format":"json","overwrite":True}}

**Explanation**
Use proper credentials and permissions for remote stores.

**Optimization options**
- Prefer object storage for large runs; rotate with timestamps.

### Batching with FEED_EXPORT_BATCH_ITEM_COUNT
Rotate output files every N items.

In [ ]:
# settings.py
FEED_EXPORT_BATCH_ITEM_COUNT = 10000
FEEDS = {"items-%(batch_id)d.json":{"format":"json","overwrite":True}}

**Explanation**
Batching keeps file sizes manageable and resilient to job restarts.

**Optimization options**
- Pick batch sizes to balance downstream throughput and failure recovery.

### Programmatic exporters
Use `ItemExporter` classes directly in a pipeline/command.

In [ ]:
from scrapy.exporters import JsonItemExporter, CsvItemExporter

class ExportPipeline:
    def open_spider(self, spider):
        self.f = open('items.json','wb')
        self.exp = JsonItemExporter(self.f, ensure_ascii=False, indent=2)
        self.exp.start_exporting()
    def close_spider(self, spider):
        self.exp.finish_exporting(); self.f.close()
    def process_item(self, item, spider):
        self.exp.export_item(item); return item

**Explanation**
Exporters let you programmatically control formatting and destinations.

**Optimization options**
- Prefer FEEDS for simple cases; use exporters when you need custom behavior.

### Custom JSON encoder
Serialize complex types by overriding encoder.

In [ ]:
from scrapy.exporters import PythonItemExporter
import json, datetime

class DateEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime.date):
            return o.isoformat()
        return json.JSONEncoder.default(self, o)

class JsonWithDatesExporter(PythonItemExporter):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._configure(kwargs, dont_fail=True)
        self.encoder = DateEncoder


**Explanation**
When items include dates/Decmial/etc., provide a custom encoder to serialize correctly.

**Optimization options**
- Normalize to strings in pipelines when possible to keep exporters simple.

### Exporting nested items cleanly
Flatten or keep structure depending on consumer.

In [ ]:
def flatten(item):
    # Transform nested dicts to flat keys
    out = {}
    for k,v in item.items():
        if isinstance(v, dict):
            for sk,sv in v.items(): out[f"{k}_{sk}"]=sv
        else:
            out[k]=v
    return out
# Use in pipeline before exporting

**Explanation**
Decide on a consistent nested/flat shape and stick to it across runs.

**Optimization options**
- Document schemas; add unit tests for transformations.

### Per-spider FEEDS via custom_settings
Define outputs without touching global settings.

In [ ]:
class MySpider(scrapy.Spider):
    name = "per_spider_feeds"
    custom_settings = {"FEEDS":{"items.csv":{"format":"csv","overwrite":True}}}

**Explanation**
Keeps experiments local to a spider.

**Optimization options**
- Avoid scattered configurations; centralize in settings for production.

### FEEDS URI params
Use feed URI parameters like Spider name/time in file paths.

In [ ]:
# settings.py
FEEDS = {"out/%(name)s-%(time)s.json":{"format":"json","overwrite":True}}

**Explanation**
`%(name)s`, `%(time)s`, `%(batch_id)d` are available placeholders.

**Optimization options**
- Use them for reproducible, unique filenames per run.

## Section 3 — Data pipelines and item processing (15 Examples)

### Pipeline skeleton
Implement `open_spider`, `process_item`, `close_spider`.

In [ ]:
import scrapy

class BasicPipeline:
    def open_spider(self, spider):
        spider.logger.info("Opening pipeline")
    def process_item(self, item, spider):
        # mutate/validate item here
        return item
    def close_spider(self, spider):
        spider.logger.info("Closing pipeline")

# settings.py
ITEM_PIPELINES = {'myproj.pipelines.BasicPipeline': 300}

**Explanation**
This is the minimal contract for an Item Pipeline.

**Optimization options**
- Keep side effects out of `process_item` unless necessary; prefer pure transforms.

### Drop invalid items
Raise `DropItem` for items that fail validation.

In [ ]:
import scrapy

class ValidatePipeline:
    def process_item(self, item, spider):
        if not item.get('title'):
            raise scrapy.exceptions.DropItem("Missing title")
        return item

**Explanation**
Dropping early avoids bad data in exporters or databases.

**Optimization options**
- Log counts of dropped reasons for QA (stats extension or custom).

### Deduplicate items (in-memory)
Track seen keys to prevent duplicates.

In [ ]:
import scrapy

class DedupePipeline:
    def open_spider(self, spider): self.seen=set()
    def process_item(self, item, spider):
        key = (item.get('title'), item.get('author'))
        if key in self.seen: raise scrapy.exceptions.DropItem("Duplicate")
        self.seen.add(key); return item

**Explanation**
Simple and fast for small jobs; memory-bound for large runs.

**Optimization options**
- For big jobs, use external stores (Redis/Bloom filter) to dedupe.

### CSV writer pipeline
Write items to CSV manually (alternative to FEEDS).

In [ ]:
import csv
class CsvPipeline:
    def open_spider(self, spider):
        self.f=open('items.csv','w',newline='',encoding='utf-8')
        self.w=csv.DictWriter(self.f, fieldnames=['title','author','tags'])
        self.w.writeheader()
    def close_spider(self, spider): self.f.close()
    def process_item(self, item, spider):
        self.w.writerow(item); return item

**Explanation**
Manual CSV gives full control over headers and ordering.

**Optimization options**
- Prefer FEEDS for simple cases; use custom CSV when you need exact control or streaming.

### SQLite pipeline
Persist items in a local SQLite DB.

In [ ]:
import sqlite3
class SQLitePipeline:
    def open_spider(self, spider):
        self.conn = sqlite3.connect('items.db'); self.cur = self.conn.cursor()
        self.cur.execute("CREATE TABLE IF NOT EXISTS t (title TEXT, author TEXT, tags TEXT)")
    def close_spider(self, spider): self.conn.commit(); self.conn.close()
    def process_item(self, item, spider):
        self.cur.execute("INSERT INTO t VALUES (?,?,?)", (item.get('title'), item.get('author'), ','.join(item.get('tags',[]))))
        return item

**Explanation**
SQLite is great for local prototypes and small projects.

**Optimization options**
- Batch commits or use `executemany` to reduce I/O overhead.

### MongoDB pipeline (pymongo)
Insert into MongoDB collection.

In [ ]:
# pip install pymongo
import pymongo
class MongoPipeline:
    def open_spider(self, spider):
        self.client = pymongo.MongoClient("mongodb://localhost:27017/")
        self.col = self.client['scrapy']['items']
    def close_spider(self, spider): self.client.close()
    def process_item(self, item, spider):
        self.col.insert_one(dict(item)); return item

**Explanation**
Mongo handles schemaless JSON-like docs; easy for nested items.

**Optimization options**
- Create indexes on query fields; avoid per-item connections.

### from_crawler to access settings
Read settings and init pipeline dynamically.

In [ ]:
from itemadapter import ItemAdapter
class ConfigurablePipeline:
    @classmethod
    def from_crawler(cls, crawler):
        inst = cls()
        inst.flag = crawler.settings.getbool('MY_PIPELINE_FLAG', False)
        return inst
    def process_item(self, item, spider):
        ad = ItemAdapter(item)
        # use inst.flag to alter behavior
        return ad.item

**Explanation**
`from_crawler` gives access to settings, signals, and crawler for dependency injection.

**Optimization options**
- Use settings to toggle behaviors without code changes.

### ItemAdapter usage
Uniform access to dicts, Items, dataclasses.

In [ ]:
from itemadapter import ItemAdapter
def as_flat_dict(item):
    ad = ItemAdapter(item)
    return {k: ad.get(k) for k in ad.field_names() or ad.keys()}

**Explanation**
ItemAdapter normalizes how you read/write fields across item types.

**Optimization options**
- Prefer ItemAdapter in shared libs/pipelines for compatibility.

### Item Loaders in pipelines
Run loader-like normalization as a pipeline transform.

In [ ]:
from itemloaders.processors import MapCompose
def strip_s(s): return (s or '').strip()
class NormalizePipeline:
    def process_item(self, item, spider):
        for k in ('title','author'):
            if k in item and isinstance(item[k], str):
                item[k] = strip_s(item[k])
        return item

**Explanation**
Simple normalization can live in pipelines; heavy parsing stays in spiders/loaders.

**Optimization options**
- Keep transforms pure; avoid network calls in pipelines.

### Media (Images) pipeline
Enable built-in `ImagesPipeline` to download item images.

In [ ]:
# settings.py
ITEM_PIPELINES = {'scrapy.pipelines.images.ImagesPipeline': 1}
IMAGES_STORE = 'images'
# Item must provide 'image_urls' list; pipeline will populate 'images'

**Explanation**
ImagesPipeline manages downloads, caching, thumbnails and more.

**Optimization options**
- Limit concurrency to image hosts; use HTTP cache to avoid re-fetching.

### Pipeline ordering and priorities
Lower numbers run first; order matters.

In [ ]:
# settings.py
ITEM_PIPELINES = {
  'myproj.pipelines.ValidatePipeline': 100,
  'myproj.pipelines.NormalizePipeline': 200,
  'myproj.pipelines.MongoPipeline': 900,
}

**Explanation**
Validate early, persist late; order transforms to avoid wasted work.

**Optimization options**
- Document pipeline order in README/settings for maintainability.

### Per-spider pipelines via custom_settings
Activate pipelines only for selected spiders.

In [ ]:
class MySpider(scrapy.Spider):
    name = "special"
    custom_settings = {
        "ITEM_PIPELINES": {'myproj.pipelines.NormalizePipeline': 200}
    }

**Explanation**
This isolates experiments without changing globals.

**Optimization options**
- Avoid configuration drift; reconcile into project defaults when stable.

### Error handling and retries inside pipelines
Catch and log exceptions; decide to drop or re-raise.

In [ ]:
import logging, scrapy
class RobustPipeline:
    def process_item(self, item, spider):
        try:
            # risky transform
            return item
        except Exception as e:
            spider.logger.exception("Pipeline error: %s", e)
            raise scrapy.exceptions.DropItem(f"Pipeline error: {e}")

**Explanation**
Fail fast on bad data with context-rich logs; do not silently swallow errors.

**Optimization options**
- Accumulate stats on failure types for QA and alerts.

### Stats & metrics from pipelines
Increment custom counters for QA.

In [ ]:
from scrapy.exceptions import DropItem
class StatsPipeline:
    def open_spider(self, spider):
        self.stats = spider.crawler.stats
    def process_item(self, item, spider):
        self.stats.inc_value('pipeline/items_seen')
        if not item.get('title'):
            self.stats.inc_value('pipeline/items_dropped/missing_title')
            raise DropItem('missing title')
        return item

**Explanation**
Use Scrapy stats to observe pipeline behavior over time.

**Optimization options**
- Hook stats into monitoring to catch regressions early.